In [12]:
import pandas as pd
import pymysql
import json
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import re

In [4]:
def setup_query():
    #connect the database
    return pymysql.connect(host="yelp.ciyx8eht0ylz.us-east-1.rds.amazonaws.com", 
                         port=3306, 
                         user="melody", 
                         password="19931206Wr.", 
                         database="yelp", 
                         charset='utf8')

In [5]:
con = setup_query()

In [6]:
q_review = """
select * from review
"""
df_review = pd.read_sql(q_review, con)

In [8]:
df_review.shape

(618150, 9)

In [7]:
df_review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,---aTjqbJIOFlMokl4KHCw,ZbvQy4QtGkeofAO_kgadCQ,m1ertlgQIxAl74D7nCsvbQ,2,0,0,0,!!!Caution!!! This is a long thorough review. ...,2019-01-13 03:48:54
1,---Q6MewoDKsudsf6HFjxA,7SgH7aavrDBjL2DbVyDGoQ,GprFEGAr8pcOF9_CgWoY8w,3,0,0,0,"The food was great, but the service was ridicu...",2010-01-12 20:51:05
2,---Z8ZHQEBD6WR2HYHmv7w,zVOQKegNfeYn7oX221GKFA,jdSwCr-q5sihRK7_ZKrt0w,5,1,0,1,I love this store. Perfect selection of small ...,2011-02-13 09:43:22
3,--05HWh3tFtXAjTV4vquHg,nt5VdYXo6k3awwPvE55xLA,uSM_sCp22BJWeatTDl6ODA,5,0,0,1,Harvest is such a hidden gem. This restaurant ...,2012-08-24 14:39:25
4,--0o-hiiY45Wx6hqNz7dJg,a5aVVq8WKIiaNoL5hvixEQ,WGZTd7vRoCeMnoqipcDoZw,1,0,0,0,"Netblazr refused to connect me, for an undiscl...",2013-07-04 19:33:07


In [13]:
#clean the text

#create a function to clean the review text
def cleanText(text):
    text = re.sub(r'https?:\/\/\S+','',text) #remove hyper link
    return text

#cleaning the text
df_review['text_clean']=df_review['text'].apply(cleanText)


In [14]:
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

#create a new column
df_review['polarity'] = df_review['text_clean'].apply(getPolarity)

In [17]:
df_out = df_review[['business_id', 'polarity']]

In [20]:
df_out_1 = df_out.groupby('business_id')['polarity'].mean().reset_index()

In [21]:
df_out_1

,business_id,polarity
0,--4INAzazK6omgf3mY3dfw,0.166670
1,--DzGwfuJH12DjYz95VW0Q,0.028775
2,--JuLhLvq3gyjNnXT9Q95w,0.299733
3,--iiZK8pW7kC2Tr2PdQrQw,0.212830
4,--nH9kKRMf25i2KREVXKtw,0.238304
...,...,...
10843,zwTNO76yQ2HsMLqBzyRmjg,0.305493
10844,zwvr3vLc36uOi5gUHi_e6Q,0.376158
10845,zxbkDCJ85JHgC8CWcdMCZw,0.273750
10846,zyTo1jeo7206-eAYkEMKiA,0.192713


In [23]:
df_out_1.to_csv('data/review_score_ma.csv',index=False)